In [1]:
using Pkg

In [2]:
Pkg.add("JuMP")

In [3]:
Pkg.add("Ipopt")

In [4]:
Pkg.resolve()

In [5]:
using JuMP
using Ipopt

In [6]:
mdl = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [11]:
mdl = Model(Ipopt.Optimizer)

@variable(mdl, x1>=0)
@variable(mdl, x2>=0)
@variable(mdl, x3>=1)

@objective(mdl, Min, x1*20 + x2*3 + 5 + x3/5)

20 x1 + 3 x2 + 0.2 x3 + 5

In [12]:
print(mdl)

In [13]:
optimize!(mdl)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [14]:
objective_value(mdl)

5.1999997755177105

In [18]:
value(x1)

0.0

In [19]:
value(x2)

0.0

In [20]:
value(x3)

1.0000000025295186

In [23]:
mdl = Model(Ipopt.Optimizer)

@variable(mdl, x1>=0)
@variable(mdl, x2>=0)
@variable(mdl, x3>=1)

a = x1*20 + x2*3 + 5 + x3/5

@objective(mdl, Min, a)

@constraint(mdl, x1+x2*x2+2>=5)

x2² + x1 >= 3.0

In [33]:
mdl2 = Model(Ipopt.Optimizer)

dmd = [200, 300, 400, 400, 440, 350]

@variable(mdl2, 200>=x1[1:6]>=70)
@variable(mdl2, 250>=x2[1:6]>=80)

@objective(mdl2, Min, sum(140*x1[i] + 0.8*x2[i]*x2[i] + 0.5*x2[i] for i in 1:6) )

a=x1+x2

dmd_cst = [] #Array{Any,1}(UndefInitializer(), 6)

for i in 1:6
    push!(dmd_cst, @constraint(mdl2, a[i] == dmd[i]))
end

In [34]:
print(mdl2)

In [35]:
optimize!(mdl2)

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       12
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [31]:
value.(x1)

6-element Vector{Float64}:
 112.81249999973444
 200.0
 200.0
 200.0
 200.0
 200.0

In [32]:
value.(x2)

6-element Vector{Float64}:
  87.18750000026554
  99.99999800017115
 199.99999800001947
 199.99999800001947
 239.9999980000144
 149.9999980000349

In [36]:
dmd_cst

6-element Vector{Any}:
 x1[1] + x2[1] == 200.0
 x1[2] + x2[2] == 300.0
 x1[3] + x2[3] == 400.0
 x1[4] + x2[4] == 400.0
 x1[5] + x2[5] == 440.0
 x1[6] + x2[6] == 350.0

In [37]:
dmd_cst[2]

x1[2] + x2[2] == 300.0

In [38]:
value(dmd_cst[2])

300.0

In [ ]:
value(dmd_cst[2])

In [40]:
dmd_testar = Array{Any,1}(UndefInitializer(), 6)

6-element Vector{Any}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

In [41]:
dmd_testar[2] = 2
dmd_testar

6-element Vector{Any}:
 #undef
   2
 #undef
 #undef
 #undef
 #undef

In [42]:
Pkg.add("Cbc")

In [43]:
using Cbc

In [47]:
mdl3 = Model(Cbc.Optimizer)

dmd2 = [400 , 320, 420, 500]

@variable(mdl3, 200>=x1[1:4]>=70)
@variable(mdl3, 250>=x2[1:4]>=80)
@variable(mdl3, 150>=x3[1:4]>=40)

#y1[1]*x1[1] + x2[1] + x3[1]

@variable(mdl3, y1[1:4], Bin)
@variable(mdl3, y2[1:4], Bin)
@variable(mdl3, y3[1:4], Bin)

@objective(mdl3, Min, sum(
        (2000+150*x1[i])+
        (2500+100*x2[i])+
        (170*80+170*x3[i])
        for i in 1:4))

@expression(mdl3, expr1, y1[1]*x1[1])
dmd_cst2 = [] #Array{Any,1}(UndefInitializer(), 6)
for i in 1:4
    push!(dmd_cst2, @constraint(mdl3, x1[i]+x2[i]+x3[i]==dmd2[i]))
end


In [48]:
optimize!(mdl3)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 201400 - 0.00 seconds
Cgl0004I processed model has 4 rows, 12 columns (0 integer (0 of which binary)) and 12 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 201400 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
FlowCover was tried 0 times 